In [29]:
!python -V

Python 3.10.12


In [30]:
import pandas as pd

In [31]:
import pickle

In [32]:
import seaborn as sns
import matplotlib.pyplot as plt

In [33]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [34]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/home/jigar/DataTalksClub/mlops-zoomcamp/02-experiment-tracking/mlruns/1', creation_time=1758616733396, experiment_id='1', last_update_time=1758616733396, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [35]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [36]:
df_train = read_dataframe('./data/green_tripdata_2025-01.parquet')
df_val = read_dataframe('./data/green_tripdata_2025-02.parquet')

In [37]:
len(df_train), len(df_val)

(46307, 44218)

In [38]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [39]:
categorical = ['PU_DO'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [40]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

2025/09/23 10:57:56 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'e641f2e3f9d14ac99ba81a08d4cd6977', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


2025/09/23 10:57:57 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'


6.098706133427926

In [43]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump(lr, f_out)

with open('models/preprocessor.bin', 'wb') as f_out:
    pickle.dump(dv, f_out)

In [44]:
from os import name
from mlflow.models import infer_signature

with mlflow.start_run():

    mlflow.set_tag("developer", "jigar")

    mlflow.log_param("train-data-path", "./data/green_tripdata_2021-01.csv")
    mlflow.log_param("valid-data-path", "./data/green_tripdata_2021-02.csv")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

2025/09/23 10:58:30 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'


In [21]:
import xgboost as xgb

In [23]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [24]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [ ]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("developer", "jigar")    # e.g.
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

mlflow.xgboost.autolog()

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=5,#50,
    trials=Trials()
)

In [26]:
mlflow.xgboost.autolog(disable=True)

In [ ]:
mlflow.xgboost.autolog()

# mlflow.autolog(log_models=True, log_input_examples=True, log_model_signatures=True)

with mlflow.start_run(run_name="xgb-classifier"):

    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42
    }

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

/home/ubuntu/software/anaconda/anaconda3/envs/mlops-zoomcamp-env/lib/python3.13/site-packages/xgboost/callback.py:386: UserWarning: [16:23:03] WARNING: /workspace/src/objective/regression_obj.cu:245: reg:linear is now deprecated in favor of reg:squarederror.
  self.starting_round = model.num_boosted_rounds()


[0]	validation-rmse:8.56745
[1]	validation-rmse:8.10033
[2]	validation-rmse:7.69739
[3]	validation-rmse:7.35105
[4]	validation-rmse:7.05239
[5]	validation-rmse:6.80020
[6]	validation-rmse:6.58162
[7]	validation-rmse:6.39726
[8]	validation-rmse:6.24197
[9]	validation-rmse:6.11059
[10]	validation-rmse:6.00193
[11]	validation-rmse:5.90700
[12]	validation-rmse:5.82752
[13]	validation-rmse:5.76123
[14]	validation-rmse:5.70306
[15]	validation-rmse:5.65560
[16]	validation-rmse:5.61619
[17]	validation-rmse:5.58065
[18]	validation-rmse:5.55157
[19]	validation-rmse:5.52734
[20]	validation-rmse:5.50858
[21]	validation-rmse:5.48872
[22]	validation-rmse:5.47379
[23]	validation-rmse:5.45957
[24]	validation-rmse:5.44705
[25]	validation-rmse:5.43706
[26]	validation-rmse:5.42713
[27]	validation-rmse:5.41891
[28]	validation-rmse:5.41283
[29]	validation-rmse:5.40819
[30]	validation-rmse:5.40244
[31]	validation-rmse:5.39854
[32]	validation-rmse:5.39446
[33]	validation-rmse:5.39004
[34]	validation-rmse:5.3

2025/09/19 16:23:22 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/19 16:23:22 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/home/ubuntu/software/anaconda/anaconda3/envs/mlops-zoomcamp-env/lib/python3.13/site-packages/mlflow/xgboost/__init__.py:169: UserWarning: [16:23:22] WARNING: /workspace/src/c_api/c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats."


In [54]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.svm import LinearSVR

# mlflow.sklearn.autolog()
mlflow.autolog(log_models=True, log_input_examples=True, log_model_signatures=True)

for model_class in (RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor, LinearSVR):
    print(f"Running {model_class.__name__}")

    with mlflow.start_run():

        mlflow.log_artifact("models/preprocessor.bin", artifact_path="preprocessor")

        mlmodel = model_class()
        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)


2025/09/23 11:06:00 INFO mlflow.tracking.fluent: Autologging successfully enabled for statsmodels.


2025/09/23 11:06:00 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.


Running GradientBoostingRegressor


2025/09/23 11:06:01 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'


Running ExtraTreesRegressor


2025/09/23 11:06:13 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'


Running LinearSVR


2025/09/23 11:15:29 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
/home/jigar/dataScienceEnv/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/home/jigar/dataScienceEnv/lib/python3.10/site-packages/sklearn/svm/_base.py:1242: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


### Model + Preprocessor

In [ ]:
import mlflow
import mlflow.sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

# Load data
X, y = load_iris(return_X_y=True, as_frame=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Fit preprocessor
scaler = StandardScaler()
scaler.fit(X_train)

# Log preprocessor
with mlflow.start_run(run_name="preprocessor-run") as run:
    mlflow.sklearn.log_model(scaler, "scaler")
    preprocessor_run_id = run.info.run_id
    print("Logged scaler in run:", preprocessor_run_id)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Apply preprocessing manually
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train model
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train_scaled, y_train)

# Log model
with mlflow.start_run(run_name="model-run") as run:
    mlflow.sklearn.log_model(clf, name="rf_model")
    mlflow.log_metric("accuracy", accuracy_score(y_test, clf.predict(X_test_scaled)))
    model_run_id = run.info.run_id
    print("Logged model in run:", model_run_id)

In [ ]:
from mlflow.sklearn import load_model

# URIs to logged artifacts
scaler_uri = f"runs:/{preprocessor_run_id}/scaler"
model_uri = f"runs:/{model_run_id}/rf_model"

# Load both
loaded_scaler = load_model(scaler_uri)
loaded_model = load_model(model_uri)

# Use for inference
X_test_scaled = loaded_scaler.transform(X_test)
preds = loaded_model.predict(X_test_scaled)

print("Loaded model accuracy:", accuracy_score(y_test, preds))

### ML Exp using mlflow client

In [ ]:
from mlflow.tracking import MlflowClient

model_name = "nyc-taxi-regressor"
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [56]:
client.create_experiment(name="my_new_exp")

'2'

In [61]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids="1",
    filter_string="metrics.rmse < 6.0",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [62]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: ca7714f02d614885972dcbafe6fcdca4, rmse: 5.6228
run id: 2d111ef190434811ae9cb168056a717d, rmse: 5.6383
run id: f933c26b98914d81a9ec2b0f9bf4551e, rmse: 5.8406


In [ ]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [65]:
run_id = "f933c26b98914d81a9ec2b0f9bf4551e"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '4' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1758620078905, current_stage='None', description=None, last_updated_timestamp=1758620078905, name='nyc-taxi-regressor', run_id='f933c26b98914d81a9ec2b0f9bf4551e', run_link=None, source='/home/jigar/DataTalksClub/mlops-zoomcamp/02-experiment-tracking/mlruns/1/f933c26b98914d81a9ec2b0f9bf4551e/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [69]:
client.get_latest_versions(name="nyc-taxi-regressor")

/tmp/ipykernel_2282324/2438216786.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.get_latest_versions(name="nyc-taxi-regressor")


[<ModelVersion: aliases=[], creation_timestamp=1758620078905, current_stage='None', description=None, last_updated_timestamp=1758620078905, name='nyc-taxi-regressor', run_id='f933c26b98914d81a9ec2b0f9bf4551e', run_link=None, source='/home/jigar/DataTalksClub/mlops-zoomcamp/02-experiment-tracking/mlruns/1/f933c26b98914d81a9ec2b0f9bf4551e/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>]

In [ ]:
# Set tag and alias
client.set_registered_model_alias(name=model_name, alias="hopeful", version=4)
client.set_model_version_tag(name=model_name, key="model", value="etr", version=4)

In [ ]:
# Or transition model stage (old mlflow model registry)
client.transition_model_version_stage(name=model_name, version=4, stage="Staging", archive_existing_versions=False)

/tmp/ipykernel_2282324/3826602534.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(name="nyc-taxi-regressor", version=4, stage="Staging", archive_existing_versions=False)


<ModelVersion: aliases=['hopeful'], creation_timestamp=1758620078905, current_stage='Staging', description=None, last_updated_timestamp=1758621499907, name='nyc-taxi-regressor', run_id='f933c26b98914d81a9ec2b0f9bf4551e', run_link=None, source='/home/jigar/DataTalksClub/mlops-zoomcamp/02-experiment-tracking/mlruns/1/f933c26b98914d81a9ec2b0f9bf4551e/artifacts/model', status='READY', status_message=None, tags={'model': 'etr'}, user_id=None, version=4>

In [ ]:
# Add description

from datetime import datetime

date = datetime.today().date()

client.update_model_version(
    name=model_name,
    version=4,
    description=f"This model was transitioned to Staging on {date}"
    )

<ModelVersion: aliases=['hopeful'], creation_timestamp=1758620078905, current_stage='Staging', description='This model was transitioned to Staging on 2025-09-23', last_updated_timestamp=1758622002863, name='nyc-taxi-regressor', run_id='f933c26b98914d81a9ec2b0f9bf4551e', run_link=None, source='/home/jigar/DataTalksClub/mlops-zoomcamp/02-experiment-tracking/mlruns/1/f933c26b98914d81a9ec2b0f9bf4551e/artifacts/model', status='READY', status_message=None, tags={'model': 'etr'}, user_id=None, version=4>

In [ ]:
# Push version 4 to production stage and the existing prod version to archive
client.transition_model_version_stage(
    name=model_name,
    version=4,
    stage="Production",
    archive_existing_versions=True
)

/tmp/ipykernel_2282324/2566576819.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=['hopeful'], creation_timestamp=1758620078905, current_stage='Production', description='This model was transitioned to Staging on 2025-09-23', last_updated_timestamp=1758622371878, name='nyc-taxi-regressor', run_id='f933c26b98914d81a9ec2b0f9bf4551e', run_link=None, source='/home/jigar/DataTalksClub/mlops-zoomcamp/02-experiment-tracking/mlruns/1/f933c26b98914d81a9ec2b0f9bf4551e/artifacts/model', status='READY', status_message=None, tags={'model': 'etr'}, user_id=None, version=4>

In [91]:
client.update_model_version(
    name=model_name,
    version=4,
    description=f"This model was transitioned to Production on {date}"
    )

<ModelVersion: aliases=['hopeful'], creation_timestamp=1758620078905, current_stage='Production', description='This model was transitioned to Production on 2025-09-23', last_updated_timestamp=1758622507836, name='nyc-taxi-regressor', run_id='f933c26b98914d81a9ec2b0f9bf4551e', run_link=None, source='/home/jigar/DataTalksClub/mlops-zoomcamp/02-experiment-tracking/mlruns/1/f933c26b98914d81a9ec2b0f9bf4551e/artifacts/model', status='READY', status_message=None, tags={'model': 'etr'}, user_id=None, version=4>